In [1]:
# importing libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [5]:
# Loading the document
loader = PyPDFLoader('The_Constitution_of_Kenya_2010.pdf')
pdf = loader.load()

In [6]:
# Embed the chunks
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeds = embeddings.embed_documents([chunk.page_content for chunk in pdf])

In [10]:
# Store in vector database
from langchain.vectorstores import FAISS
db = FAISS.from_documents(pdf, embeddings)

%run groq_api.ipynb
import os
os.environ["GROQ_API_KEY"] = qroq_api_key
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
qroq_llm = ChatGroq(model_name="moonshotai/kimi-k2-instruct-0905", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=qroq_llm, chain_type="stuff", 
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True)

In [23]:
# Test the model
query = "How many elective posts are in Kenya?"
result = qa_chain({"query": query})
print("Answer:", result['result'])
print("Source Document:", result['source_documents'])


Answer: The Constitution of Kenya, 2010 creates six (6) categories of elective posts:

1. President and Deputy President  
2. Members of Parliament (290 constituency MPs + 47 women county MPs + 12 nominated MPs)  
3. Senators (47 elected + 20 nominated)  
4. County Governors and their Deputy Governors (47)  
5. Members of County Assemblies – MCAs (1,450 elected + 772 nominated)  
6. Woman Representatives to the County Assemblies (47)  

Hence, there are six distinct elective posts in Kenya.
Source Document: [Document(id='e10397e1-2df4-4da4-ae5c-cddd031e60ac', metadata={'producer': 'LibreOffice 3.6', 'creator': 'Writer', 'creationdate': '2013-02-28T17:50:04+03:00', 'title': 'Rev. 2010]', 'author': 'Andare Jeffrey', 'source': 'The_Constitution_of_Kenya_2010.pdf', 'total_pages': 193, 'page': 8, 'page_label': '9'}, page_content='Constitution of Kenya, 2010\n216—Functions of the Commission on Revenue Allocation.\n217—Division of revenue.\n218—Annual Division and Allocation of Revenue Bills.

In [17]:
!pip install gradio
import gradio as gr

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/60.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/60.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/60.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/60.4 MB ? eta -:--:--
    --------------------------------------- 1.0/60.4 MB 2.8 MB/s eta 0:00:22
   - -------------------------------------- 2.6/60.4 MB 4.4 MB/s eta 0:00:14
   -- ------------------------------------- 4.5/60.4 MB 5.7 MB/s eta 0:00:10
   -- ------------------------------------- 4.5/60.4 MB 5.7 MB/s eta 0:00:10
   -- ------------------------------------- 4.5/60.4 MB 5.7 MB/s eta 0:00:10
   -- ------------------------------------- 4.5/60.4 MB 5.7 MB/s eta 0:00:10
   --- ------------------------------------ 4.7/60.4 MB 3.1 MB/s eta 0:00:18
   --- ------------------------------------ 5.0/60.4 MB 2.6 MB/s eta 0:00:22
   --- -------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [28]:
import gradio as gr
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

# Groq API key
%run groq_api.ipynb
os.environ["GROQ_API_KEY"] = qroq_api_key

# retriever
qroq_llm = ChatGroq(model_name="moonshotai/kimi-k2-instruct-0905", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=qroq_llm, chain_type="stuff", retriever=db.as_retriever())

def ask(query):
    return qa_chain({"query": query})["result"]

gr.Interface(fn=ask, inputs=gr.Textbox(lines=2), outputs=gr.Textbox(lines=2), title="Ask the Constitution").launch(share=True)


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://06315f0fec874eec0a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
